In [1]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_ollama import ChatOllama

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
llm = ChatOllama(model="deepseek-coder-v2")

In [4]:
llm.invoke("Tell me a joke")

AIMessage(content=" Why don't scientists trust atoms? Because they make up everything!", additional_kwargs={}, response_metadata={'model': 'deepseek-coder-v2', 'created_at': '2025-02-12T10:11:20.8482236Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8331683200, 'load_duration': 7850683400, 'prompt_eval_count': 12, 'prompt_eval_duration': 211000000, 'eval_count': 15, 'eval_duration': 267000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-65454ab7-10c0-49a2-b190-c779afccbfb6-0', usage_metadata={'input_tokens': 12, 'output_tokens': 15, 'total_tokens': 27})

In [5]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [6]:
db_name=os.environ["DB_DATABASE"]
print(db_name)
db_user = os.environ["DB_USERNAME"]
db_password = os.environ["DB_PASSWORD"]
db_host = "localhost"
db_port = 3307
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

naseni


In [7]:
from langchain.chains import create_sql_query_chain
chain = create_sql_query_chain(llm,db)

In [8]:
from mysql.connector import ProgrammingError

In [9]:
import re

In [10]:
def extract_sql_query(text):
    # Define a regex pattern to capture SQL queries
    sql_pattern = re.compile(r"(SELECT\s[\s\S]+?;)", re.IGNORECASE)
    
    # Search for the pattern in the text
    match = sql_pattern.findall(text)
    return [query.strip() for query in match][0]


In [11]:
def execute_query(question):
    try:
        response = chain.invoke({"question": question})
        response = re.sub(r'<think>.*?</think>', '', response,flags=re.DOTALL).strip()
        query = extract_sql_query(response)
        print(query)
        result = db.run(query)
        print(result)
    except ProgrammingError as e:
        print(f"An error occurred: {e}")

In [12]:
execute_query("I want all the proposals table name is proposal_submission")

C:\Users\Ali\Documents\taha projects\GenAI\Taha\L4(Databese)\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "ntb_companies, ntb_memo, page_comments, signature_logs", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
C:\Users\Ali\Documents\taha projects\GenAI\Taha\L4(Databese)\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "ntb_companies, ntb_memo, page_comments, signature_logs", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an e

ResponseError: an error was encountered while running the model: read tcp 127.0.0.1:57600->127.0.0.1:57598: wsarecv: An existing connection was forcibly closed by the remote host. (status code: -1)